In [1]:
# Import libraries 
import pandas as pd
import numpy as np
import os
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Get working directory
current_dir = os.getcwd()
#go one directory up to root directory
project_root_dir = os.path.dirname(current_dir)
#Define path to data files
data_dir = os.path.join(project_root_dir, 'data')
raw_dir = os.path.join(data_dir, 'raw')
processed_dir = os.path.join(data_dir, 'processed')
#Define path to results folder
results_dir = os.path.join(project_root_dir, 'results')
#Define path to results folder
docs_dir = os.path.join(project_root_dir, 'docs')

#Create directories if they do not exist
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)
os.makedirs(docs_dir, exist_ok=True)

In [5]:
video_filename = os.path.join(raw_dir, "lex_fridman_videos.csv")
video_df = pd.read_csv(video_filename, na_values="?", skipinitialspace=True)

comments_filename = os.path.join(raw_dir, "lex_fridman_comments.csv")
comments_df = pd.read_csv(comments_filename, na_values="?", skipinitialspace=True)

In [7]:
print(video_df.columns)
print(comments_df.columns)

Index(['videoId', 'title', 'publishedAt', 'viewCount', 'likeCount',
       'commentCount', 'tags', 'description'],
      dtype='object')
Index(['videoId', 'authorDisplayName', 'textDisplay', 'likeCount',
       'publishedAt'],
      dtype='object')


In [9]:
print(video_df.shape)
print(comments_df.shape)

(94, 8)
(4684, 5)


In [11]:
print(video_df.info())
print("---------------------------------------------------")
print(comments_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   videoId       94 non-null     object
 1   title         94 non-null     object
 2   publishedAt   94 non-null     object
 3   viewCount     94 non-null     int64 
 4   likeCount     94 non-null     int64 
 5   commentCount  94 non-null     int64 
 6   tags          94 non-null     object
 7   description   91 non-null     object
dtypes: int64(3), object(5)
memory usage: 6.0+ KB
None
---------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4684 entries, 0 to 4683
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   videoId            4684 non-null   object
 1   authorDisplayName  4682 non-null   object
 2   textDisplay        4682 non-null   object
 3   likeCount          4684 n

In [13]:
print(video_df.isna().sum())
print("---------------------------------")
print(comments_df.isna().sum())

videoId         0
title           0
publishedAt     0
viewCount       0
likeCount       0
commentCount    0
tags            0
description     3
dtype: int64
---------------------------------
videoId              0
authorDisplayName    2
textDisplay          2
likeCount            0
publishedAt          0
dtype: int64


In [15]:
video_df['description'] = video_df['description'].fillna('Null')
comments_df['authorDisplayName'] = comments_df['authorDisplayName'].fillna('unknown')

In [17]:
print(video_df.isna().sum())
print("---------------------------------")
print(comments_df.isna().sum())

videoId         0
title           0
publishedAt     0
viewCount       0
likeCount       0
commentCount    0
tags            0
description     0
dtype: int64
---------------------------------
videoId              0
authorDisplayName    0
textDisplay          2
likeCount            0
publishedAt          0
dtype: int64


In [19]:
print(video_df.duplicated().sum())
print("---------------------------------")
print(comments_df.duplicated().sum())

0
---------------------------------
0


In [21]:
print(video_df.dtypes == object)
print("---------------------------------")
print(comments_df.dtypes == object)

videoId          True
title            True
publishedAt      True
viewCount       False
likeCount       False
commentCount    False
tags             True
description      True
dtype: bool
---------------------------------
videoId               True
authorDisplayName     True
textDisplay           True
likeCount            False
publishedAt           True
dtype: bool


In [23]:
video_df.head()

,videoId,title,publishedAt,viewCount,likeCount,commentCount,tags,description
0,UN5qgBk6MwY,Terence Tao on Grigori Perelman declining the ...,2025-06-19T23:23:00Z,167222,3301,60,[],Null
1,HUkBz-cdB-k,"Terence Tao: Hardest Problems in Mathematics, ...",2025-06-14T20:15:34Z,702226,15133,1426,"['Terence Tao', 'alex friedman', 'lex ai', 'le...",Terence Tao is widely considered to be one of ...
2,b268x2kzefY,Lex Fridman tests Google XR glasses,2025-06-13T00:37:30Z,89074,2819,147,[],Null
3,ufI31PNM9u8,"Google CEO on Elon Musk, Tesla, and Waymo",2025-06-10T20:51:39Z,78715,1805,90,[],Null
4,sXPONXB8WCo,Lex Fridman tests Google Beam,2025-06-06T23:04:59Z,401132,6607,261,[],This is a clip from a podcast episode with Goo...


In [25]:
comments_df.head()

,videoId,authorDisplayName,textDisplay,likeCount,publishedAt
0,UN5qgBk6MwY,@iliya-malecki,Id prefer to watch a video on one of the great...,0,2025-06-25T23:29:49Z
1,UN5qgBk6MwY,@4D_art,Grigori “Pearl”man .,0,2025-06-25T20:48:16Z
2,UN5qgBk6MwY,@tomorrows-med,Pure genius,0,2025-06-25T14:04:09Z
3,UN5qgBk6MwY,@extavwudda,I am so sick of Lex sucking up to people,0,2025-06-25T06:08:24Z
4,UN5qgBk6MwY,@michealvallieres9228,Dude why would you interview a guy that's neve...,0,2025-06-23T23:40:40Z


## Clean Video Metadata

Here’s what we’ll do:

- Convert dates to datetime

- Make numbers numeric

- Clean text fields

In [27]:
# Convert publishedAt to datetime
video_df['publishedAt'] = pd.to_datetime(video_df['publishedAt'])

# Convert numeric fields
video_df['viewCount'] = pd.to_numeric(video_df['viewCount'], errors='coerce')
video_df['likeCount'] = pd.to_numeric(video_df['likeCount'], errors='coerce')
video_df['commentCount'] = pd.to_numeric(video_df['commentCount'], errors='coerce')

# Clean text fields (remove line breaks and extra spaces)
video_df['title'] = video_df['title'].str.replace('\n', ' ').str.strip()
video_df['description'] = video_df['description'].str.replace('\n', ' ').str.strip()

video_df.head()

,videoId,title,publishedAt,viewCount,likeCount,commentCount,tags,description
0,UN5qgBk6MwY,Terence Tao on Grigori Perelman declining the ...,2025-06-19 23:23:00+00:00,167222,3301,60,[],Null
1,HUkBz-cdB-k,"Terence Tao: Hardest Problems in Mathematics, ...",2025-06-14 20:15:34+00:00,702226,15133,1426,"['Terence Tao', 'alex friedman', 'lex ai', 'le...",Terence Tao is widely considered to be one of ...
2,b268x2kzefY,Lex Fridman tests Google XR glasses,2025-06-13 00:37:30+00:00,89074,2819,147,[],Null
3,ufI31PNM9u8,"Google CEO on Elon Musk, Tesla, and Waymo",2025-06-10 20:51:39+00:00,78715,1805,90,[],Null
4,sXPONXB8WCo,Lex Fridman tests Google Beam,2025-06-06 23:04:59+00:00,401132,6607,261,[],This is a clip from a podcast episode with Goo...


In [29]:
video_df.columns

Index(['videoId', 'title', 'publishedAt', 'viewCount', 'likeCount',
       'commentCount', 'tags', 'description'],
      dtype='object')

In [33]:
video_df.rename(columns={'publishedAt': 'videoPublishedAt'}, inplace=True)
comments_df.rename(columns={'publishedAt': 'commentPublishedAt'}, inplace=True)
video_df.rename(columns={'likeCount': 'videoLikeCount'}, inplace=True)
comments_df.rename(columns={'likeCount': 'commentLikeCount'}, inplace=True)

In [37]:
print(video_df.columns)
print(comments_df.columns)

Index(['videoId', 'title', 'videoPublishedAt', 'viewCount', 'videoLikeCount',
       'commentCount', 'tags', 'description'],
      dtype='object')
Index(['videoId', 'authorDisplayName', 'textDisplay', 'commentLikeCount',
       'commentPublishedAt'],
      dtype='object')


In [39]:
# Convert publishedAt to datetime
video_df['videoPublishedAt'] = pd.to_datetime(video_df['videoPublishedAt'])

# Convert numeric fields
video_df['viewCount'] = pd.to_numeric(video_df['viewCount'], errors='coerce')
video_df['videoLikeCount'] = pd.to_numeric(video_df['videoLikeCount'], errors='coerce')
video_df['commentCount'] = pd.to_numeric(video_df['commentCount'], errors='coerce')

# Clean text fields (remove line breaks and extra spaces)
video_df['title'] = video_df['title'].str.replace('\n', ' ').str.strip()
video_df['description'] = video_df['description'].str.replace('\n', ' ').str.strip()

video_df.head()

,videoId,title,videoPublishedAt,viewCount,videoLikeCount,commentCount,tags,description
0,UN5qgBk6MwY,Terence Tao on Grigori Perelman declining the ...,2025-06-19 23:23:00+00:00,167222,3301,60,[],Null
1,HUkBz-cdB-k,"Terence Tao: Hardest Problems in Mathematics, ...",2025-06-14 20:15:34+00:00,702226,15133,1426,"['Terence Tao', 'alex friedman', 'lex ai', 'le...",Terence Tao is widely considered to be one of ...
2,b268x2kzefY,Lex Fridman tests Google XR glasses,2025-06-13 00:37:30+00:00,89074,2819,147,[],Null
3,ufI31PNM9u8,"Google CEO on Elon Musk, Tesla, and Waymo",2025-06-10 20:51:39+00:00,78715,1805,90,[],Null
4,sXPONXB8WCo,Lex Fridman tests Google Beam,2025-06-06 23:04:59+00:00,401132,6607,261,[],This is a clip from a podcast episode with Goo...


## Clean Comment Data

Here’s what we’ll do:

- Remove empty or deleted comments

- Convert date

- Clean and tokenize text

In [44]:
# Remove [deleted] or empty comments
comments_df = comments_df[comments_df['textDisplay'].notna()]
comments_df = comments_df[~comments_df['textDisplay'].str.contains('\[deleted\]', case=False)]

# Convert to datetime
comments_df['commentPublishedAt'] = pd.to_datetime(comments_df['commentPublishedAt'])

# Clean comment text
def clean_comment(text):
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"[^A-Za-z\s]", "", text)  # Remove special characters
    text = text.lower()
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')]
    return " ".join(words)

comments_df['clean_text'] = comments_df['textDisplay'].astype(str).apply(clean_comment)

comments_df.head()

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:3: SyntaxWarning: invalid escape sequence '\['
C:\Users\user\AppData\Local\Temp\ipykernel_3160\3211246972.py:3: SyntaxWarning: invalid escape sequence '\['
  comments_df = comments_df[~comments_df['textDisplay'].str.contains('\[deleted\]', case=False)]


,videoId,authorDisplayName,textDisplay,commentLikeCount,commentPublishedAt,clean_text
0,UN5qgBk6MwY,@iliya-malecki,Id prefer to watch a video on one of the great...,0,2025-06-25 23:29:49+00:00,id prefer watch video one greatest minds human...
1,UN5qgBk6MwY,@4D_art,Grigori “Pearl”man .,0,2025-06-25 20:48:16+00:00,grigori pearlman
2,UN5qgBk6MwY,@tomorrows-med,Pure genius,0,2025-06-25 14:04:09+00:00,pure genius
3,UN5qgBk6MwY,@extavwudda,I am so sick of Lex sucking up to people,0,2025-06-25 06:08:24+00:00,sick lex sucking people
4,UN5qgBk6MwY,@michealvallieres9228,Dude why would you interview a guy that's neve...,0,2025-06-23 23:40:40+00:00,dude would interview guy thats never met right...


In [46]:
print(comments_df.isna().sum())

videoId               0
authorDisplayName     0
textDisplay           0
commentLikeCount      0
commentPublishedAt    0
clean_text            0
dtype: int64


In [48]:
comments_df.shape

(4682, 6)

## Merge the datasets

- Purpose: Combines both datasets using `videoId` as the key.

- inner join ensures only matching records from both datasets are kept.

In [54]:
merged_df = pd.merge(comments_df, video_df, on="videoId", how="inner")
merged_df.head()

,videoId,authorDisplayName,textDisplay,commentLikeCount,commentPublishedAt,clean_text,title,videoPublishedAt,viewCount,videoLikeCount,commentCount,tags,description
0,UN5qgBk6MwY,@iliya-malecki,Id prefer to watch a video on one of the great...,0,2025-06-25 23:29:49+00:00,id prefer watch video one greatest minds human...,Terence Tao on Grigori Perelman declining the ...,2025-06-19 23:23:00+00:00,167222,3301,60,[],Null
1,UN5qgBk6MwY,@4D_art,Grigori “Pearl”man .,0,2025-06-25 20:48:16+00:00,grigori pearlman,Terence Tao on Grigori Perelman declining the ...,2025-06-19 23:23:00+00:00,167222,3301,60,[],Null
2,UN5qgBk6MwY,@tomorrows-med,Pure genius,0,2025-06-25 14:04:09+00:00,pure genius,Terence Tao on Grigori Perelman declining the ...,2025-06-19 23:23:00+00:00,167222,3301,60,[],Null
3,UN5qgBk6MwY,@extavwudda,I am so sick of Lex sucking up to people,0,2025-06-25 06:08:24+00:00,sick lex sucking people,Terence Tao on Grigori Perelman declining the ...,2025-06-19 23:23:00+00:00,167222,3301,60,[],Null
4,UN5qgBk6MwY,@michealvallieres9228,Dude why would you interview a guy that's neve...,0,2025-06-23 23:40:40+00:00,dude would interview guy thats never met right...,Terence Tao on Grigori Perelman declining the ...,2025-06-19 23:23:00+00:00,167222,3301,60,[],Null


In [56]:
merged_df.duplicated().sum()

0

In [58]:
merged_df.isna().sum()

videoId               0
authorDisplayName     0
textDisplay           0
commentLikeCount      0
commentPublishedAt    0
clean_text            0
title                 0
videoPublishedAt      0
viewCount             0
videoLikeCount        0
commentCount          0
tags                  0
description           0
dtype: int64

In [60]:
clean_filename = os.path.join(processed_dir, "Video_Comments_DS.csv")
merged_df.to_csv(clean_filename, index=False)
print("Saved Video&Comments. Total:", len(merged_df))
print(f"\nCleaned data saved to: {clean_filename}")

Saved Video&Comments. Total: 4682

Cleaned data saved to: C:\Users\user\Documents\tekHer\YouTube-Channel-Insights\data\processed\Video_Comments_DS.csv
